# Fastbin Double Free Attack


## Example: samsara

Source: MetasequoiaCTF 202002020-0221



In [4]:
bin_filename = 'fastbin-double-free/samsara'
from pwn import *
from pwnlib import gdb

context.terminal = ['tmux', 'new-window']
elf = ELF(bin_filename)
print(elf.checksec())

RELRO:    Full RELRO
Stack:    Canary found
NX:       NX enabled
PIE:      PIE enabled


In [6]:
gdb.debug(bin_filename)

[x] Starting local process '/usr/bin/gdbserver'
[+] Starting local process '/usr/bin/gdbserver': pid 173
[*] running in new terminal: /usr/bin/gdb -q  "fastbin-double-free/samsara" -x /tmp/pwnw4k5tt0q.gdb


In [13]:
io = process(bin_filename)
io.recvline()


[x] Starting local process 'fastbin-double-free/samsara'
[+] Starting local process 'fastbin-double-free/samsara': pid 199


b'After defeating the Demon Dragon, you turned yourself into the Demon Dragon...\n'